In [2]:
import pandas as pd 

In [7]:
data_dir_path = '../data/'


In [25]:
def get_nyc_zcta5():
    zcta5_file_name = 'zip_to_zcta10_nyc_revised.xls'
    zcta5_df = pd.read_excel(data_dir_path + zcta5_file_name)
    zcta5_list = [str(x) for x in set(zcta5_df['zcta5'])]
    return zcta5_list
zcta5_list = get_nyc_zcta5()
print('length of zcta5_list: ', len(zcta5_list))
print('zcta5_list[:10]: ', zcta5_list[:10])

length of zcta5_list:  214
zcta5_list[:10]:  ['10271', '10278', '10279', '10280', '10282', '10301', '10302', '10303', '10304', '10305']


In [46]:
def get_nyc_only_df(input_df, zcta5_list):
    nyc_geo_list = ['8600000US' + x for x in zcta5_list]
    print('nyc_geo_list[:10]: ', nyc_geo_list[:10])
    # print(input_df['GEO_ID'].isin(nyc_geo_list))
    nyc_only_df = input_df[input_df['GEO_ID'].isin(nyc_geo_list)]
    nyc_only_df.index = range(len(nyc_only_df)) # set the index from 0
    print()
    print('nyc_only_df rows: ', nyc_only_df.shape[0])
    return nyc_only_df

### Ratio of women to men (all ages)

In [47]:
# read data
B05003_dir_path = 'ACSDT5Y2018.B05003_2020-04-15T173748/'
B05003_file_name = 'ACSDT5Y2018.B05003_data_with_overlays_2020-04-15T173740.csv'
B05003_df = pd.read_csv(data_dir_path + B05003_dir_path + B05003_file_name)

# get nyc only data frame
B05003_nyc_only_df = get_nyc_only_df(B05003_df, zcta5_list)

# get the needed features(columns)
male_num_feature = 'B05003_002E' # Estimate!!Total!!Male
female_num_feature = 'B05003_013E' # Estimate!!Total!!Female

# result data frame
result_df = pd.DataFrame(columns=['GEO_ID', 'NAME', 'Ratio_women_to_men'])
for i, row in B05003_nyc_only_df.iterrows():
    try:
        result_df.loc[i] = [row['GEO_ID'], row['NAME'], row[female_num_feature]/row[male_num_feature]]
    except:
        # print(i)
        # print('row[female_num_feature]: ', row[female_num_feature])
        # print('row[male_num_feature]: ', row[male_num_feature])
        result_df.loc[i] = [row['GEO_ID'], row['NAME'], 'None'] # Both row[female_num_feature] and row[male_num_feature] is 0

result_df

/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214


,GEO_ID,NAME,Ratio_women_to_men
0,8600000US11222,ZCTA5 11222,1.02733
1,8600000US11233,ZCTA5 11233,1.18882
2,8600000US11235,ZCTA5 11235,1.0872
3,8600000US11426,ZCTA5 11426,1.00241
4,8600000US11427,ZCTA5 11427,1.0223
...,...,...,...
209,8600000US11218,ZCTA5 11218,0.984869
210,8600000US11223,ZCTA5 11223,1.03896
211,8600000US11224,ZCTA5 11224,1.23169
212,8600000US11373,ZCTA5 11373,0.997683
